# **Automated program for TESTING by using YOLOv8 framework and trained model on custom dataset**

In [ ]:
# connect to your Google drive
from google.colab import drive
drive.mount('/content/drive')

# change to the YOLOv8 or your project directory
# mine was Yolov8_id_main
%cd /content/drive/MyDrive/Yolov8_id_main/

# **ELA MODULE**

In [ ]:
# required packages for working on ELA module
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools
import cv2
import glob

In [ ]:
# give path of your colored and tampered images
real_image_path = '/content/drive/MyDrive/Yolov8_id_main/test/real_image/'

# give the path where the converted ELA images of colored tampered images will be saved
ELA_output_path = '/content/drive/MyDrive/Yolov8_id_main/test/ELA_image/'

# give the path where the colored and tampered images with bounding boxes will be saved after gone through YOLOv8 module
predicted_image_path = '/content/drive/MyDrive/Yolov8_id_main/test/predicted_image/'

In [ ]:
# Function for converting colored and tampered images into ELA images

def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'

    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)

    ela_image = ImageChops.difference(image, temp_image)

    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff

    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)

    return ela_image

In [ ]:
# converting all real images into ELA images
for img in glob.glob(real_image_path + "*.*"):
  conv_img = convert_to_ela_image(img, 90)
  base_name = os.path.basename(img)
  conv_img.save(os.path.join(ELA_output_path, base_name))

# **COPY PASTE TAMPERING BOUNDARY ARTIFACTS DETECTION MODULE (YOLOv8)**

In [ ]:
# installing required packages for working on YOLOv8
!pip install ultralytics
from ultralytics import YOLO

# load your trained model
model = YOLO("weights.pt")

In [ ]:
# function for drawing bounding boxes on colored and tampered images depending
# upon the bounding boxes detected by YOLOv8
def create_refined_bounding_boxes(image, detections):
    """
    Creates refined bounding boxes around individual detections in an image.

    Args:
        image (numpy.ndarray): The image as a numpy array.
        detections (list): A list of detection bounding boxes. Each detection should be a list/tuple
                          of four values: [x_min, y_min, x_max, y_max].

    Returns:
        tuple: A tuple containing:
            - The image with the refined bounding boxes drawn on it.
            - A list of tuples, where each tuple represents the coordinates of a refined bounding box
              (x_min, y_min, x_max, y_max).
              Returns an empty list if no detections are provided.
    """
    refined_boxes = []
    # checking if there are any detections
    if not detections:
        print("No detections provided. Returning original image and empty list.")
        return image, refined_boxes

    # iterating through the detections and draw individual bounding boxes
    for x_min, y_min, x_max, y_max in detections:
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 4)  # Red rectangle, thickness 2
        refined_boxes.append((x_min, y_min, x_max, y_max))

    return image, refined_boxes

In [ ]:
# function for detecting copy paste boundary artifacts on ELA image by using YOLOv8
# after detecting it will draw bounding boxes on colored and tampered image

for img in glob.glob(ELA_output_path + "*.*"):
  # Perform prediction with YOLOv8
  results = model.predict(source=img, show=False, save=True, conf=0.5)

  # checking for bounding boxes -
  # if exists then it is forgery image
  # else no forgery had been done on that image
  for result in results:
    if len(result.boxes) > 0:
      print("Forgery detected: Bounding boxes found.")
      break  # Exit the loop after the first detection
    else:
      print("No forgery detected: No bounding boxes found.")

  # extracting the base name
  base_name = os.path.basename(img)
  # taking the colored and tampered image with same base name
  realImg = real_image_path + base_name
  # Load the image using cv2
  image = cv2.imread(realImg)

  # extracting bounding box coordinates from YOLOv8 results
  detections = []
  for result in results:
    for box in result.boxes:
      x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Convert to integers
      detections.append((x_min, y_min, x_max, y_max))

  # creating the refined bounding boxes
  image_with_boxes, refined_boxes = create_refined_bounding_boxes(image.copy(), detections)  # Pass a copy of the image

  # saving the image with the refined bounding boxes
  if image_with_boxes is not None:
    cv2.imwrite(predicted_image_path + base_name, image_with_boxes)